# 2️⃣번째 기능 - 냉장고 재료 비중이 많은 음식을 우선으로 추천
 code_all_model 파일에서 전처리하고 조리법 및 난이도를 라벨링한 최종 데이터를
입력 데이터 기준 유사한 음식/냉장고 재료 비중이 많은/기피 재료가 최소을 기준으로 원하는 레시피를 검색하는 코드 흐름

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터 다운
#final_data=pd.read_csv('/content/drive/My Drive/DataCampus/project/FINAL_DATA.csv', encoding='cp949', index_col=0)
#final_data=pd.read_excel('/content/drive/MyDrive/동덕여대/2023-2강의자료/빅데이터캡스톤/만개의레시피_다이어트메뉴데이터(2000개).xlsx')
#final_data=pd.read_excel('/content/drive/MyDrive/동덕여대/2023-2강의자료/빅데이터캡스톤/만개의레시피_전처리1차완료(정은).xlsx')
final_data=pd.read_excel('/content/drive/MyDrive/동덕여대/2023-2강의자료/빅데이터캡스톤/만개의레시피_전처리1차완료(정은).xlsx')
final_data


,Unnamed: 0,메뉴명,설명,재료,레시피,해시태그
0,0,맛있는 도토리묵무침 만드는 법 다이어트음식으로 좋아요,노출의 계절인 여름을 앞두고 본격적인 다이어트에 돌입한 분들에게 추천하고픈 다이어트...,"도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루 ...","1. 묵은 플라스틱 케이스에서 꺼내 반으로 잘라줍니다., 2. 도토리묵을 끓는 물에...","다이어트, 다이어트음식, 초간단, 저칼로리, 야채, 도토리묵, 도토리묵무침, 무침,..."
1,1,맛있는 다이어트 도토리묵 무침 만들기,딱 봄까지만 관리 해보자며 다욧을 빡시게 하고 있지만 역시 먹는걸 참는건 너무 힘드...,"도토리묵 500, 양파 1/2개, 오이 1개, 상추 5장, 당근 적당량, 고춧가루 ...",1. 도토리묵은 끓는물에 1~2분간 데쳐 주셔요~ 전 1키로 짜리지만 반만(500g...,"다이어트, 도토리묵, 도토리묵무침, 묵무침, 간단반찬, 저칼로리, 초간단, 술안주,..."
2,2,다이어트 술안주 생오징어쪽파숙회,제철인 생오징어를 햇쪽파를 데쳐서 같이 곁들여보세요 담백하면서도 부드럽고 쫄깃한맛으...,"오징어 2마리, 쪽파 1주먹, 초장 조금, 소금 1작은술, 생강술 1큰술, 밀가루3...",1. 생오징어를 준비해주세요 *냉동오징어를 이용하실경우 자연해동시켜주신후 조리하세요...,"다이어트반찬, 아이들반찬, 초간단, 건강식단, 손님접대음식, 집들이음식, 술안주"
3,3,포케 샐러드 포케소스 만들기 다이어트 식단 아이올리 소스 만드는법 병아리콩샐러드 새...,오늘 요리는 다이어트에 도움이 되는 하와이식 회덮밥 포케 입니다. 참치나 연어를 넣...,"샐러드 믹스 70, 토마토 1개, 크린베리 10, 삶은 병아리콩 1줌, 구운 아몬드...",1. 냉동 칵테일 새우를 사용하였어요. 요리하기 10분 전 찬물에 담가 냉동 새우를...,"포케, 포케소스, 포케만들기, 새우샐러드, 병아리콩샐러드, 아이올리소스, 다이어트식..."
4,4,다이어트요리 두부카레볶음밥,초간단하게 만들수있는 다이어트 볶음밥!,"두부 1모, 계란 1개, 우유 5숫갈, 부추 조금, 카레가루 1스푼반, 소금 조금","1. 먼저 두부는 물기를 따라내고 숫가락으로 대충 으깨 주시구요., 2. 카레한스푼...",태그 정보 없음
...,...,...,...,...,...,...
2085,2085,미역줄기볶음 이보은 레시피 국민밑반찬 아이반찬 어른반찬 두가지 버전 비린내제거 맛술...,미역줄거리볶음 국민밑반찬이지만 비린내로 싫어하신다면 맛술에 10분만 절인 후 볶아보세요,"손질한 미역 줄기 300, 대파 흰부분 3T, 양파 1/2개, 당근 약간, 식용유 ...",1. 염장한 미역 줄거리를 물에 행구어 소금을 제거한 후 바락바락 주물러 3~4번 ...,"미역줄기볶음, 밑반찬, 맛술, 비린내제거"
2086,2086,알토란 구기자코다리찜 코다리찜 코다리콩나물찜 코다리요리 구기자 멸치육수 가을철 대표...,매콤한 양념에 아삭한 콩나물과 코다리를 즐길 수 있는 코다리찜 만들어 보세요!!,"코다리 750, 굵은 콩나물 200, 구기자 30, 무 100, 대파 1개, 청양고...","1. 다시 멸치의 까만 내장을 제거한 후, 달군 팬에 살짝 바삭하게 구워준다., 2...","코다리찜, 코다리콩나물찜, 코다리"
2087,2087,두부달걀볶음밥 두부요리 달걀요리 다이어트메뉴 채식식단 두부소보로 한그릇요리 초간단요...,다이어트 할 때 닭가슴살에 질리셨다면 두부와 달걀을 이용한 두부달걀볶음밥 한 번 만...,"두부 1모, 달걀 3개, 맛소금 약간, 대파 1대, 밥 2숟가락, 간장 1T, 식용...","1. 대파는 흰 부분과 초록부분을 반으로 가른 뒤 송송 썬다., 2. 달걀 2~3개...","다이어트, 두부, 달걀, 볶음밥, 두부달걀볶음밥, 채식, 건강식, 단백질"
2088,2088,메밀샐러드 5분다이어트요리,샐러드만 먹기 지겹고 배고프다면! 메밀면을 함께,"양상추 1/4통, 파프리카 1/2개, 오리엔탈소스 2~4스푼, 봉평촌 메밀국수 10...","1. 봉평촌 메밀국수 100g을 끓는물에서 3분정도 삶습니다., 2. 메밀면이 잘 ...","다이어트, 샐러드, 5분요리, 초간단, 메밀, 메밀면, 저녁메뉴, 점심메뉴, 오리엔..."


In [5]:
import re

def clean_ingredients(text):


    ingredients = text.split(', ')

    cleaned_ingredients = []
    for ingredient in ingredients:
        # 숫자가 있는 경우 숫자가 시작하는 지점을 찾음
        match = re.search(r'\d', ingredient)
        if match:
            start_index = match.start()
            cleaned_ingredients.append(ingredient[:start_index])
        else:
            cleaned_ingredients.append(ingredient)

    final_ingredients = [ingredient.strip() for ingredient in cleaned_ingredients if ingredient.strip()]
    return final_ingredients

text = "양파1/4개, 소금, 양조간장1가지 선택"
result = clean_ingredients(text)
print(result)


['양파', '소금', '양조간장']


In [6]:
import re

def clean_ingredients(text):
    text = text.replace('약간', '').replace('조금', '')
    pattern = r'\s*\([^)]*\)\s*'
    text = re.sub(pattern, '', text)
    text = text.replace('\u200b', '')  # '\u200b' 제거
    ingredients = text.split(', ')

    cleaned_ingredients = []
    for ingredient in ingredients:
        # 숫자가 있는 경우 숫자가 시작하는 지점을 찾음
        match = re.search(r'\d', ingredient)
        if match:
            start_index = match.start()
            cleaned_ingredients.append(ingredient[:start_index])
        else:
            cleaned_ingredients.append(ingredient)

    final_ingredients = [ingredient.strip() for ingredient in cleaned_ingredients if ingredient.strip()]
    return final_ingredients

def text_preprocessor2(s):
    s_list = clean_ingredients(s)
    if '재료 정보 없음' in s_list:
        return '재료 정보 없음'
    return s_list


In [7]:
## sample text
s = '도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루'

## apply the UDF above
s_list = text_preprocessor2(s)
print(s_list)

['도토리묵', '상추', '깻잎', '당근', '고춧가루']


In [8]:
for ingr in final_data['재료'][:5]:
  print(text_preprocessor2(ingr))


['도토리묵', '상추', '깻잎', '당근', '고춧가루', '진간장', '매실청', '설탕', '통깨', '대파', '청양고추']
['도토리묵', '양파', '오이', '상추', '당근 적당량', '고춧가루', '진간장', '설탕', '식초', '다진마늘', '참기름']
['오징어', '쪽파', '초장', '소금', '생강술', '밀가루']
['샐러드 믹스', '토마토', '크린베리', '삶은 병아리콩', '구운 아몬드', '냉동 아보카도', '블랙 올리브', '냉동 새우', '올리브유', '버터', '소금', '후춧가루', '올리브유', '발사믹 식초', '꿀', '다진 마늘', '후춧가루']
['두부', '계란', '우유', '부추', '카레가루', '소금']


In [9]:
#재료마다 컬럼명이 떨어져 있는 형태의 버전이라면 이것을 사용
'''after = pd.DataFrame(columns=["전처리 후 재료"])

# 반복문을 통해 데이터 추가
for ingr in final_data['재료']:
    ingr_data=text_preprocessor(ingr)  # 각 행의 데이터
    after = after.append(pd.Series(ingr_data), ignore_index=True)'''

'after = pd.DataFrame(columns=["전처리 후 재료"])\n\n# 반복문을 통해 데이터 추가\nfor ingr in final_data[\'재료\']:\n    ingr_data=text_preprocessor(ingr)  # 각 행의 데이터\n    after = after.append(pd.Series(ingr_data), ignore_index=True)'

In [10]:
after = pd.DataFrame(columns=["전처리 후 재료"])

# 반복문을 통해 데이터 추가
for ingr in final_data['재료']:
    ingredient_list=text_preprocessor2(ingr)  # 각 행의 데이터
    if ingredient_list!='재료 정보 없음':
      ingredient_str = ', '.join(ingredient_list)
    else:
      ingredient_str=ingredient_list
    after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)

<ipython-input-10-827da8ad4a2b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)
<ipython-input-10-827da8ad4a2b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)
<ipython-input-10-827da8ad4a2b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)
<ipython-input-10-827da8ad4a2b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=aft

In [11]:
final_data['재료'][:10]

0    도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루 ...
1    도토리묵 500, 양파 1/2개, 오이 1개, 상추 5장, 당근 적당량, 고춧가루 ...
2    오징어 2마리, 쪽파 1주먹, 초장 조금, 소금 1작은술, 생강술 1큰술, 밀가루3...
3    샐러드 믹스 70, 토마토 1개, 크린베리 10, 삶은 병아리콩 1줌, 구운 아몬드...
4        두부 1모, 계란 1개, 우유 5숫갈, 부추 조금, 카레가루 1스푼반, 소금 조금
5    양배추 1/4쪽, 애호박 1개, 당근 1개, 두부 1모, 계란 2개, 라이스페이퍼 ...
6                                             재료 정보 없음
7    두부 1모, 계란 1개, 팽이버섯 1봉, 양파 1개, 당근 1/2개, 대파 1대, ...
8    양배추 400, 당근 1/2개, 계란 3개, 식빵 3장, 소금 1/2스푼, 오일 2...
9    두부 1/2모, 청양고추 1개, 양파 1/2개, 다진마늘 0.5수저, 된장 2수저,...
Name: 재료, dtype: object

In [12]:
import pandas as pd

def rearrange_columns(final_data, after):
    # 데이터프레임을 복사하여 새로운 데이터프레임 생성
    save_data = final_data.copy()

    # '재료' 열을 '재료명' 열로 복사
    save_data['재료명'] = save_data['재료']

    # '재료단위' 열을 '전처리 후 재료' 열로 복사
    save_data['재료단위'] = after['전처리 후 재료']

    # '재료' 열 제거
    save_data.drop('재료', axis=1, inplace=True)

    # 컬럼 순서 재배열
    save_data = save_data[['메뉴명', '설명', '재료단위', '재료명', '레시피', '해시태그']]

    # 데이터프레임을 반환
    return save_data

# 함수 호출
rearranged_data = rearrange_columns(final_data, after)

# 데이터프레임을 엑셀 파일로 내보내기
rearranged_data.to_excel('재료명단위제거output_1024.xlsx', index=False)


In [13]:
after.head()

,전처리 후 재료
0,"도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, ..."
1,"도토리묵, 양파, 오이, 상추, 당근 적당량, 고춧가루, 진간장, 설탕, 식초, 다..."
2,"오징어, 쪽파, 초장, 소금, 생강술, 밀가루"
3,"샐러드 믹스, 토마토, 크린베리, 삶은 병아리콩, 구운 아몬드, 냉동 아보카도, 블..."
4,"두부, 계란, 우유, 부추, 카레가루, 소금"


In [14]:
import pandas as pd
before=pd.DataFrame(final_data['재료'])
before.head()

,재료
0,"도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루 ..."
1,"도토리묵 500, 양파 1/2개, 오이 1개, 상추 5장, 당근 적당량, 고춧가루 ..."
2,"오징어 2마리, 쪽파 1주먹, 초장 조금, 소금 1작은술, 생강술 1큰술, 밀가루3..."
3,"샐러드 믹스 70, 토마토 1개, 크린베리 10, 삶은 병아리콩 1줌, 구운 아몬드..."
4,"두부 1모, 계란 1개, 우유 5숫갈, 부추 조금, 카레가루 1스푼반, 소금 조금"


In [15]:
result = pd.DataFrame(pd.concat([before, after], axis=1))

# 결과 출력
result

,재료,전처리 후 재료
0,"도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루 ...","도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, ..."
1,"도토리묵 500, 양파 1/2개, 오이 1개, 상추 5장, 당근 적당량, 고춧가루 ...","도토리묵, 양파, 오이, 상추, 당근 적당량, 고춧가루, 진간장, 설탕, 식초, 다..."
2,"오징어 2마리, 쪽파 1주먹, 초장 조금, 소금 1작은술, 생강술 1큰술, 밀가루3...","오징어, 쪽파, 초장, 소금, 생강술, 밀가루"
3,"샐러드 믹스 70, 토마토 1개, 크린베리 10, 삶은 병아리콩 1줌, 구운 아몬드...","샐러드 믹스, 토마토, 크린베리, 삶은 병아리콩, 구운 아몬드, 냉동 아보카도, 블..."
4,"두부 1모, 계란 1개, 우유 5숫갈, 부추 조금, 카레가루 1스푼반, 소금 조금","두부, 계란, 우유, 부추, 카레가루, 소금"
...,...,...
2085,"손질한 미역 줄기 300, 대파 흰부분 3T, 양파 1/2개, 당근 약간, 식용유 ...","손질한 미역 줄기, 대파 흰부분, 양파, 당근, 식용유, 맛술, 생수, 진간장, 국..."
2086,"코다리 750, 굵은 콩나물 200, 구기자 30, 무 100, 대파 1개, 청양고...","코다리, 굵은 콩나물, 구기자, 무, 대파, 청양고추, 홍고추, 찹쌀가루, 물, 통..."
2087,"두부 1모, 달걀 3개, 맛소금 약간, 대파 1대, 밥 2숟가락, 간장 1T, 식용...","두부, 달걀, 맛소금, 대파, 밥, 간장, 식용유"
2088,"양상추 1/4통, 파프리카 1/2개, 오리엔탈소스 2~4스푼, 봉평촌 메밀국수 10...","양상추, 파프리카, 오리엔탈소스, 봉평촌 메밀국수"


In [17]:
result_data=pd.DataFrame(pd.concat([final_data['메뉴명'],after['전처리 후 재료']], axis=1))
result_data

,메뉴명,전처리 후 재료
0,맛있는 도토리묵무침 만드는 법 다이어트음식으로 좋아요,"도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, ..."
1,맛있는 다이어트 도토리묵 무침 만들기,"도토리묵, 양파, 오이, 상추, 당근 적당량, 고춧가루, 진간장, 설탕, 식초, 다..."
2,다이어트 술안주 생오징어쪽파숙회,"오징어, 쪽파, 초장, 소금, 생강술, 밀가루"
3,포케 샐러드 포케소스 만들기 다이어트 식단 아이올리 소스 만드는법 병아리콩샐러드 새...,"샐러드 믹스, 토마토, 크린베리, 삶은 병아리콩, 구운 아몬드, 냉동 아보카도, 블..."
4,다이어트요리 두부카레볶음밥,"두부, 계란, 우유, 부추, 카레가루, 소금"
...,...,...
2085,미역줄기볶음 이보은 레시피 국민밑반찬 아이반찬 어른반찬 두가지 버전 비린내제거 맛술...,"손질한 미역 줄기, 대파 흰부분, 양파, 당근, 식용유, 맛술, 생수, 진간장, 국..."
2086,알토란 구기자코다리찜 코다리찜 코다리콩나물찜 코다리요리 구기자 멸치육수 가을철 대표...,"코다리, 굵은 콩나물, 구기자, 무, 대파, 청양고추, 홍고추, 찹쌀가루, 물, 통..."
2087,두부달걀볶음밥 두부요리 달걀요리 다이어트메뉴 채식식단 두부소보로 한그릇요리 초간단요...,"두부, 달걀, 맛소금, 대파, 밥, 간장, 식용유"
2088,메밀샐러드 5분다이어트요리,"양상추, 파프리카, 오리엔탈소스, 봉평촌 메밀국수"


In [18]:
tfidf = TfidfVectorizer()
# tf-idf 수행
tfidf_matrix = tfidf.fit_transform(result_data['전처리 후 재료'])
print(tfidf_matrix.shape)

(2090, 2011)


In [19]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [22]:
# 재료가 유사한 레시피 검색하는 함수 정의
def get_recommendations(cosine_sim=cosine_sim):

  ingredient=input('받아올 재료를 입력하세요(쉼표로 재료 구분) : ')

  ingredient=', '.join(ingredient.split(' '))
  #print(ingredient)

  #search_data=data.append(pd.DataFrame(data={'num':[1275], 'title':['USER'], 'ingredient': [ingredient]}))

  search_data=pd.concat([result_data.loc[:, ['메뉴명', '전처리 후 재료']], pd.DataFrame(data={'메뉴명':['USER'], '전처리 후 재료': [ingredient]})], axis=0)
  #print(search_data)
  #print(len(search_data))

  tfidf = TfidfVectorizer()
  # INGREDIENT 에 대해서 tf-idf 수행
  tfidf_matrix = tfidf.fit_transform(search_data['전처리 후 재료'])
  #print(tfidf_matrix.shape)

  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

  sim_scores = list(enumerate(cosine_sim[(len(search_data)-1)]))

  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  sim_scores = sim_scores[1:11]

  movie_indices = [i[0] for i in sim_scores]

  return result_data['메뉴명'].iloc[movie_indices]


In [23]:
get_recommendations()

받아올 재료를 입력하세요(쉼표로 재료 구분) : 닭고기, 양상추


1039                        수비드닭가슴살_전기밥솥으로 간단하게 만드는 다이어트식
1514                 청양고추를 넣은 닭가슴살 고소한 버터 볶음 다이어트 닭가슴살 볶음
404               올 여름 다이어트 끝장 레시피 닭가슴살 냉채 맛있게 먹고 다이어트 하기
669                               누구든 다이어트음식으로 먹을수있는 월남쌈 
1093                              다이어트  최애반찬 닭가슴살소세지야채볶음 
1075                      햄 대신 닭고기 넣은 오므라이스 만드는 법 다이어트 식단
825                               다이어트도 맛있게 먹으면서 생선카레볶음밥 
1056                            상큼 담백한 다이어트 식단 크래미샐러드 레시피
1939     시원한음식 샐러드 라면 만들기  다이어트  건강  간단  파스타 대신  냉라면 셀...
866                                   다이어트 레시피  닭가슴살 양상추쌈
Name: 메뉴명, dtype: object

# 이전 작성한 함수

In [ ]:
'''def text_preprocessor(s): #출처:https://rfriend.tistory.com/748
    import re

    ## (1) [], (), {}, <> 괄호와 괄호 안 문자 제거하기
    pattern = r'\([^)]*\)'  # ()
    s = re.sub(pattern=pattern, repl='', string=s)

    pattern = r'\[[^)]*\]'  # []
    s = re.sub(pattern=pattern, repl='', string=s)

    pattern = r'\<[^)]*\>'  # <>
    s = re.sub(pattern=pattern, repl='', string=s)

    pattern = r'\{[^)]*\}'  # {}
    s = re.sub(pattern=pattern, repl='', string=s)



    ## (2) '약간','조금' 제거하기
    s = s.replace('약간', '')
    s = s.replace('조금', '')

    ## (3) 특수문자 제거
    pattern = r'[^a-zA-Z가-힣]'
    s = re.sub(pattern=pattern, repl=' ', string=s)


    # (4) 공백 기준으로 분할하기
    s_split = s.split(', ')
    print('s:' ,s)
    print('s_split:' ,s_split)
    ## (5) 단위 제거:
    # (6) 재료 정보 없으면 제외하기
    s_list = []
    for word in s_split:
        if word=='재료 정보 없음':
          return
        else:
          for word in s_split:
            if len(word.split(' ')[:-1])!=0:
              #one_ingredient=word.split(' ')[:-1]
              #print(' '.join(word))
              word_split=word.split(' ')[:-1]
              one_ingredient=' '.join(word_split)
              print('one_ingredient:' ,one_ingredient)
            else:
              one_ingredient=word
              print('one_ingredient:',one_ingredient)
              s_list.append(one_ingredient)
          return s_list'''

"def text_preprocessor(s): #출처:https://rfriend.tistory.com/748\n    import re\n\n    ## (1) [], (), {}, <> 괄호와 괄호 안 문자 제거하기\n    pattern = r'\\([^)]*\\)'  # ()\n    s = re.sub(pattern=pattern, repl='', string=s)\n\n    pattern = r'\\[[^)]*\\]'  # []\n    s = re.sub(pattern=pattern, repl='', string=s)\n\n    pattern = r'\\<[^)]*\\>'  # <>\n    s = re.sub(pattern=pattern, repl='', string=s)\n\n    pattern = r'\\{[^)]*\\}'  # {}\n    s = re.sub(pattern=pattern, repl='', string=s)\n\n\n\n    ## (2) '약간','조금' 제거하기\n    s = s.replace('약간', '')\n    s = s.replace('조금', '')\n\n    ## (3) 특수문자 제거\n    pattern = r'[^a-zA-Z가-힣]'\n    s = re.sub(pattern=pattern, repl=' ', string=s)\n\n\n    # (4) 공백 기준으로 분할하기\n    s_split = s.split(', ')\n    print('s:' ,s)\n    print('s_split:' ,s_split)\n    ## (5) 단위 제거:\n    # (6) 재료 정보 없으면 제외하기\n    s_list = []\n    for word in s_split:\n        if word=='재료 정보 없음':\n          return\n        else:\n          for word in s_split:\n            if len(word.spli

In [ ]:
s = '손질한 미역 줄기 300g, 대파 흰부분 3T, 양파 1/2개, 당근 약간, 식용유'
s_split = s.split(', ')
print('s_split:',s_split)
s_list = []
for word in s_split:
  if len(word.split(' ')[:-1])!=0:
    #one_ingredient=word.split(' ')[:-1]
    #print(' '.join(word))
    word=word.split(' ')[:-1]
    one_ingredient=' '.join(word)

  else:
    one_ingredient=word
  print('one_ingredient:',one_ingredient)
  s_list.append(one_ingredient)

s_list

s_split: ['손질한 미역 줄기 300g', '대파 흰부분 3T', '양파 1/2개', '당근 약간', '식용유']
one_ingredient: 손질한 미역 줄기
one_ingredient: 대파 흰부분
one_ingredient: 양파
one_ingredient: 당근
one_ingredient: 식용유


['손질한 미역 줄기', '대파 흰부분', '양파', '당근', '식용유']

In [ ]:
def text_preprocessor(s): #출처:https://rfriend.tistory.com/748
    import re

    ## (2) '약간','조금' 제거하기
    s = s.replace('약간', '')
    s = s.replace('조금', '')

    #(3) 괄호안 문자 제거
    pattern = r'\s*\([^)]*\)\s*'

    # 정규 표현식을 사용하여 괄호 안의 내용을 제거
    s = re.sub(pattern, '', s)

    # (4) 공백 기준으로 분할하기
    s_split = s.split(', ')
    #print('s:' ,s)
    #print('s_split:' ,s_split)
    ## (5) 단위 제거:
    # (6) 재료 정보 없으면 제외하기
    s_list = []
    for word in s_split:
        if word=='재료 정보 없음':
          return '재료 정보 없음'
        else:
          if len(word.split(' ')[:-1])!=0:
              #one_ingredient=word.split(' ')[:-1]
              #print(' '.join(word))
              word_split=word.split(' ')[:-1]
              one_ingredient=' '.join(word_split)
              #print('one_ingredient:' ,one_ingredient)
          else:
              one_ingredient=word
              #print('one_ingredient:',one_ingredient)
          s_list.append(one_ingredient)
    return s_list

In [ ]:
## sample text
s = '도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루'

## apply the UDF above
s_list = text_preprocessor(s)
print(s_list)

['도토리묵', '상추', '깻잎', '당근', '고춧가루']


In [ ]:
for ingr in final_data['재료'][:5]:
  print(text_preprocessor(ingr))


['도토리묵', '상추', '깻잎', '당근', '고춧가루', '진간장', '매실청', '설탕', '통깨', '대파', '청양고추1개']
['도토리묵', '양파', '오이', '상추', '당근', '고춧가루', '진간장', '설탕', '식초', '다진마늘', '참기름']
['오징어', '쪽파', '초장', '소금', '생강술', '밀가루3~4큰술']
['샐러드 믹스', '토마토', '크린베리', '삶은 병아리콩', '구운 아몬드', '냉동 아보카도', '블랙 올리브', '냉동 새우', '\u200b 올리브유', '버터', '소금', '후춧가루', '\u200b 올리브유', '발사믹 식초', '꿀', '다진 마늘', '후춧가루']
['두부', '계란', '우유', '부추', '카레가루', '소금']


In [ ]:
ingredient_list = ['도토리묵', '상추', '깻잎', '당근', '고춧가루', '진간장', '매실청', '설탕', '통깨', '대파', '청양고추1개']
ingredient_str = ', '.join(ingredient_list)
print(ingredient_str)


도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, 청양고추1개


In [ ]:
#재료마다 컬럼명이 떨어져 있는 형태의 버전이라면 이것을 사용
'''after = pd.DataFrame(columns=["전처리 후 재료"])

# 반복문을 통해 데이터 추가
for ingr in final_data['재료']:
    ingr_data=text_preprocessor(ingr)  # 각 행의 데이터
    after = after.append(pd.Series(ingr_data), ignore_index=True)'''

'after = pd.DataFrame(columns=["전처리 후 재료"])\n\n# 반복문을 통해 데이터 추가\nfor ingr in final_data[\'재료\']:\n    ingr_data=text_preprocessor(ingr)  # 각 행의 데이터\n    after = after.append(pd.Series(ingr_data), ignore_index=True)'

In [ ]:
after = pd.DataFrame(columns=["전처리 후 재료"])

# 반복문을 통해 데이터 추가
for ingr in final_data['재료']:
    ingredient_list=text_preprocessor(ingr)  # 각 행의 데이터
    if ingredient_list!='재료 정보 없음':
      ingredient_str = ', '.join(ingredient_list)
    else:
      ingredient_str=ingredient_list
    after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)

<ipython-input-16-5a1bd96352f9>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)
<ipython-input-16-5a1bd96352f9>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)
<ipython-input-16-5a1bd96352f9>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=after.columns), ignore_index=True)
<ipython-input-16-5a1bd96352f9>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  after = after.append(pd.Series(ingredient_str,index=aft

In [ ]:
final_data['재료'][:10]

0    도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루 ...
1    도토리묵 500, 양파 1/2개, 오이 1개, 상추 5장, 당근 적당량, 고춧가루 ...
2    오징어 2마리, 쪽파 1주먹, 초장 조금, 소금 1작은술, 생강술 1큰술, 밀가루3...
3    샐러드 믹스 70, 토마토 1개, 크린베리 10, 삶은 병아리콩 1줌, 구운 아몬드...
4        두부 1모, 계란 1개, 우유 5숫갈, 부추 조금, 카레가루 1스푼반, 소금 조금
5    양배추 1/4쪽, 애호박 1개, 당근 1개, 두부 1모, 계란 2개, 라이스페이퍼 ...
6                                             재료 정보 없음
7    두부 1모, 계란 1개, 팽이버섯 1봉, 양파 1개, 당근 1/2개, 대파 1대, ...
8    양배추 400, 당근 1/2개, 계란 3개, 식빵 3장, 소금 1/2스푼, 오일 2...
9    두부 1/2모, 청양고추 1개, 양파 1/2개, 다진마늘 0.5수저, 된장 2수저,...
Name: 재료, dtype: object

In [ ]:
import pandas as pd

def rearrange_columns(final_data, after):
    # 데이터프레임을 복사하여 새로운 데이터프레임 생성
    save_data = final_data.copy()

    # '재료' 열을 '재료명' 열로 복사
    save_data['재료명'] = save_data['재료']

    # '재료단위' 열을 '전처리 후 재료' 열로 복사
    save_data['재료단위'] = after['전처리 후 재료']

    # '재료' 열 제거
    save_data.drop('재료', axis=1, inplace=True)

    # 컬럼 순서 재배열
    save_data = save_data[['메뉴명', '설명', '재료단위', '재료명', '레시피', '해시태그']]

    # 데이터프레임을 반환
    return save_data

# 함수 호출
rearranged_data = rearrange_columns(final_data, after)

# 데이터프레임을 엑셀 파일로 내보내기
rearranged_data.to_excel('재료명단위제거output.xlsx', index=False)


In [ ]:
after.head()

,전처리 후 재료
0,"도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, ..."
1,"도토리묵, 양파, 오이, 상추, 당근, 고춧가루, 진간장, 설탕, 식초, 다진마늘,..."
2,"오징어, 쪽파, 초장, 소금, 생강술, 밀가루3~4큰술"
3,"샐러드 믹스, 토마토, 크린베리, 삶은 병아리콩, 구운 아몬드, 냉동 아보카도, 블..."
4,"두부, 계란, 우유, 부추, 카레가루, 소금"


In [ ]:
import pandas as pd
before=pd.DataFrame(final_data['재료'])
before.head()

,재료
0,"도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루 ..."
1,"도토리묵 500, 양파 1/2개, 오이 1개, 상추 5장, 당근 적당량, 고춧가루 ..."
2,"오징어 2마리, 쪽파 1주먹, 초장 조금, 소금 1작은술, 생강술 1큰술, 밀가루3..."
3,"샐러드 믹스 70, 토마토 1개, 크린베리 10, 삶은 병아리콩 1줌, 구운 아몬드..."
4,"두부 1모, 계란 1개, 우유 5숫갈, 부추 조금, 카레가루 1스푼반, 소금 조금"


In [ ]:
result = pd.DataFrame(pd.concat([before, after], axis=1))

# 결과 출력
result

,재료,전처리 후 재료
0,"도토리묵 1/2팩, 상추 6-7장, 깻잎 5장, 당근(색 내기용) 조금, 고춧가루 ...","도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, ..."
1,"도토리묵 500, 양파 1/2개, 오이 1개, 상추 5장, 당근 적당량, 고춧가루 ...","도토리묵, 양파, 오이, 상추, 당근, 고춧가루, 진간장, 설탕, 식초, 다진마늘,..."
2,"오징어 2마리, 쪽파 1주먹, 초장 조금, 소금 1작은술, 생강술 1큰술, 밀가루3...","오징어, 쪽파, 초장, 소금, 생강술, 밀가루3~4큰술"
3,"샐러드 믹스 70, 토마토 1개, 크린베리 10, 삶은 병아리콩 1줌, 구운 아몬드...","샐러드 믹스, 토마토, 크린베리, 삶은 병아리콩, 구운 아몬드, 냉동 아보카도, 블..."
4,"두부 1모, 계란 1개, 우유 5숫갈, 부추 조금, 카레가루 1스푼반, 소금 조금","두부, 계란, 우유, 부추, 카레가루, 소금"
...,...,...
2085,"손질한 미역 줄기 300, 대파 흰부분 3T, 양파 1/2개, 당근 약간, 식용유 ...","손질한 미역 줄기, 대파 흰부분, 양파, 당근, 식용유, 맛술, 생수, 진간장, 국..."
2086,"코다리 750, 굵은 콩나물 200, 구기자 30, 무 100, 대파 1개, 청양고...","코다리, 굵은 콩나물, 구기자, 무, 대파, 청양고추, 홍고추, 찹쌀가루, 물, 통..."
2087,"두부 1모, 달걀 3개, 맛소금 약간, 대파 1대, 밥 2숟가락, 간장 1T, 식용...","두부, 달걀, 맛소금, 대파, 밥, 간장, 식용유"
2088,"양상추 1/4통, 파프리카 1/2개, 오리엔탈소스 2~4스푼, 봉평촌 메밀국수 10...","양상추, 파프리카, 오리엔탈소스, 봉평촌 메밀국수"


In [ ]:
result_data=pd.DataFrame(pd.concat([final_data['메뉴명'],after['전처리 후 재료']], axis=1))
result_data

,메뉴명,전처리 후 재료
0,맛있는 도토리묵무침 만드는 법 다이어트음식으로 좋아요,"도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, ..."
1,맛있는 다이어트 도토리묵 무침 만들기,"도토리묵, 양파, 오이, 상추, 당근, 고춧가루, 진간장, 설탕, 식초, 다진마늘,..."
2,다이어트 술안주 생오징어쪽파숙회,"오징어, 쪽파, 초장, 소금, 생강술, 밀가루3~4큰술"
3,포케 샐러드 포케소스 만들기 다이어트 식단 아이올리 소스 만드는법 병아리콩샐러드 새...,"샐러드 믹스, 토마토, 크린베리, 삶은 병아리콩, 구운 아몬드, 냉동 아보카도, 블..."
4,다이어트요리 두부카레볶음밥,"두부, 계란, 우유, 부추, 카레가루, 소금"
...,...,...
2085,미역줄기볶음 이보은 레시피 국민밑반찬 아이반찬 어른반찬 두가지 버전 비린내제거 맛술...,"손질한 미역 줄기, 대파 흰부분, 양파, 당근, 식용유, 맛술, 생수, 진간장, 국..."
2086,알토란 구기자코다리찜 코다리찜 코다리콩나물찜 코다리요리 구기자 멸치육수 가을철 대표...,"코다리, 굵은 콩나물, 구기자, 무, 대파, 청양고추, 홍고추, 찹쌀가루, 물, 통..."
2087,두부달걀볶음밥 두부요리 달걀요리 다이어트메뉴 채식식단 두부소보로 한그릇요리 초간단요...,"두부, 달걀, 맛소금, 대파, 밥, 간장, 식용유"
2088,메밀샐러드 5분다이어트요리,"양상추, 파프리카, 오리엔탈소스, 봉평촌 메밀국수"


In [ ]:
tfidf = TfidfVectorizer()
# tf-idf 수행
tfidf_matrix = tfidf.fit_transform(result_data['전처리 후 재료'])
print(tfidf_matrix.shape)

(2090, 2216)


In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
pd.concat([result_data.loc[:, ['메뉴명', '전처리 후 재료']], pd.DataFrame(data={'메뉴명':['USER'], '전처리 후 재료': ['당근\n양파']}, index=[1274])], axis=0) #수정중

,메뉴명,전처리 후 재료
0,맛있는 도토리묵무침 만드는 법 다이어트음식으로 좋아요,"도토리묵, 상추, 깻잎, 당근, 고춧가루, 진간장, 매실청, 설탕, 통깨, 대파, ..."
1,맛있는 다이어트 도토리묵 무침 만들기,"도토리묵, 양파, 오이, 상추, 당근, 고춧가루, 진간장, 설탕, 식초, 다진마늘,..."
2,다이어트 술안주 생오징어쪽파숙회,"오징어, 쪽파, 초장, 소금, 생강술, 밀가루3~4큰술"
3,포케 샐러드 포케소스 만들기 다이어트 식단 아이올리 소스 만드는법 병아리콩샐러드 새...,"샐러드 믹스, 토마토, 크린베리, 삶은 병아리콩, 구운 아몬드, 냉동 아보카도, 블..."
4,다이어트요리 두부카레볶음밥,"두부, 계란, 우유, 부추, 카레가루, 소금"
...,...,...
2086,알토란 구기자코다리찜 코다리찜 코다리콩나물찜 코다리요리 구기자 멸치육수 가을철 대표...,"코다리, 굵은 콩나물, 구기자, 무, 대파, 청양고추, 홍고추, 찹쌀가루, 물, 통..."
2087,두부달걀볶음밥 두부요리 달걀요리 다이어트메뉴 채식식단 두부소보로 한그릇요리 초간단요...,"두부, 달걀, 맛소금, 대파, 밥, 간장, 식용유"
2088,메밀샐러드 5분다이어트요리,"양상추, 파프리카, 오리엔탈소스, 봉평촌 메밀국수"
2089,포두부파이 겉 바 속 촉 쌈두부요리 포두부요리 맜있는다이어트요리,"포두부, 계란, 양파, 방울토마토, 토마토소스, 모짜렐라치즈, 소금, 허브가루"


In [ ]:
# 재료가 유사한 레시피 검색하는 함수 정의
def get_recommendations(cosine_sim=cosine_sim):

  ingredient=input('받아올 재료를 입력하세요(쉼표로 재료 구분) : ')

  ingredient=', '.join(ingredient.split(' '))
  #print(ingredient)

  #search_data=data.append(pd.DataFrame(data={'num':[1275], 'title':['USER'], 'ingredient': [ingredient]}))

  search_data=pd.concat([result_data.loc[:, ['메뉴명', '전처리 후 재료']], pd.DataFrame(data={'메뉴명':['USER'], '전처리 후 재료': [ingredient]})], axis=0)
  #print(search_data)
  #print(len(search_data))

  tfidf = TfidfVectorizer()
  # INGREDIENT 에 대해서 tf-idf 수행
  tfidf_matrix = tfidf.fit_transform(search_data['전처리 후 재료'])
  #print(tfidf_matrix.shape)

  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

  sim_scores = list(enumerate(cosine_sim[(len(search_data)-1)]))

  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  sim_scores = sim_scores[1:11]

  movie_indices = [i[0] for i in sim_scores]

  return result_data['메뉴명'].iloc[movie_indices]


In [ ]:
get_recommendations()

KeyboardInterrupt: ignored

# 참고자료

In [ ]:
tfidf = TfidfVectorizer()
# tf-idf 수행
tfidf_matrix = tfidf.fit_transform(final_data['재료'])
print(tfidf_matrix.shape)

(2090, 2233)


In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
pd.concat([final_data.loc[:, ['메뉴명', '재료']], pd.DataFrame(data={'메뉴명':['USER'], '재료': ['당근\n양파']}, index=[1274])], axis=0) #수정중

,TITLE,ALL_INGREDIENT
0,도토리묵,도토리묵 가루\n물\n소금\n들기름\n참기름
1,아보카도 들기름 국수 샐러드,소면\n쌀국수\n파스타\n채소\n아보카도\n양파\n오이\n김\n간장\n들기름\n참깨...
2,토마토 마리네이드,방울토마토\n양파\n마늘\n올리브오일\n발사믹 식초\n레몬즙\n메이플시럽\n소금\n...
3,마늘 요거트 드레싱 샐러드,플레인요거트\n메이플 시럽\n레몬즙\n마늘\n소금\n후추\n채소\n딸기\n포도\n견...
4,버섯 샐러드,간장\n올리브오일\n발사믹식초\n레몬즙\n마늘\n생강\n메이플시럽\n후추\n버섯\n...
...,...,...
1270,콩고기부리또,또띠아\n 밥\n 콩고기\n 버섯\n 맛간장\n 치즈\n 발사믹드레싱\n 토마토살사
1271,과카몰리샐러드,아보카도\n 야채\n 견과류\n 치즈\n 토마토살사\n드레싱\n 소금\n 후추\n...
1272,토마토살사,레몬즙\n 양파\n 토마토\n 파프리카\n 올리브유\n 설탕\n 소금
1273,야채가득떡국,애호박\n 당근\n 표고버섯\n 양파\n 느타리버섯\n 대파\n 떡국떡\n 배\n ...


In [ ]:
# 재료가 유사한 레시피 검색하는 함수 정의
def get_recommendations(cosine_sim=cosine_sim):

  ingredient=input('받아올 재료를 입력하세요(공백으로 재료 구분) : ')

  ingredient='\n'.join(ingredient.split(' '))
  #print(ingredient)

  #search_data=data.append(pd.DataFrame(data={'num':[1275], 'title':['USER'], 'ingredient': [ingredient]}))

  search_data=pd.concat([final_data.loc[:, ['TITLE', 'ALL_INGREDIENT']], pd.DataFrame(data={'TITLE':['USER'], 'ALL_INGREDIENT': [ingredient]})], axis=0)
  #print(search_data)
  #print(len(search_data))

  tfidf = TfidfVectorizer()
  # INGREDIENT 에 대해서 tf-idf 수행
  tfidf_matrix = tfidf.fit_transform(search_data['ALL_INGREDIENT'])
  #print(tfidf_matrix.shape)

  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

  sim_scores = list(enumerate(cosine_sim[(len(search_data)-1)]))

  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  sim_scores = sim_scores[1:11]

  movie_indices = [i[0] for i in sim_scores]

  return final_data['TITLE'].iloc[movie_indices]


In [ ]:
get_recommendations()

받아올 재료를 입력하세요(공백으로 재료 구분) : 고구마 감자 양파 파


324                                  토마토 카레
726                                     야채칩
950                                   고구마맛탕
835                                칼륨듬뿍고구마죽
376                  청양고추와 병아리콩을 이용한 매콤채식카레
910                                 호박고구마스프
401                                    감자주스
975    고구마맛탕만들기, 메이플시럽으로 풍미를 더해보세요 \n고구마 맛탕
424                                   고구마빠스
723                                 야채과일돌구이
Name: TITLE, dtype: object